In [20]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Input, BatchNormalization, ReLU, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt

import PIL
import os
from os import listdir

In [2]:
class ResNetBlock(keras.Model):
    def __init__(self, filters, kernel_size = (3,3), strides=1, downsample = False, strides_2 = None):
        super().__init__()
        if strides_2 is None : strides_2 = strides
        self.conv1 = Conv2D(filters, kernel_size, strides = strides, padding = "same")
        self.norm = BatchNormalization()
        self.relu = ReLU()
        self.conv2 = Conv2D(filters, kernel_size, strides = strides_2, padding = "same")
        self.norm2 = BatchNormalization()
        self.downsample = downsample
        if downsample ==True:
            self.downsample_conv = Conv2D(filters, 1, strides)
            self.downsample_norm = BatchNormalization()
        else:
            self.downsample_conv = None
            self.downsample_norm = None
    
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.norm(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        y = inputs
        if self.downsample:
            y = self.downsample_conv(y)
            y = self.downsample_norm(y)
        #print(x.shape, y.shape)
        x = x + y
        x = self.relu(x)
        return x

In [3]:
inputs = Input((224, 224,3))
x = Conv2D(64, (7,7), strides = 2, padding="same")(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((2,2))(x)


x = ResNetBlock(64)(x) #Block_1
x = ResNetBlock(64)(x) #Block_2
x = ResNetBlock(64)(x) #Block_3

x = ResNetBlock(128, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(128)(x) #Block_5
x = ResNetBlock(128)(x) #Block_6
x = ResNetBlock(128)(x) #Block_7

x = ResNetBlock(256, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(256)(x) #Block_9
x = ResNetBlock(256)(x) #Block_10
x = ResNetBlock(256)(x) #Block_11
x = ResNetBlock(256)(x) #Block_12
x = ResNetBlock(256)(x) #Block_13

x = ResNetBlock(512, strides = 2, downsample = True, strides_2 = 1)(x) #Block_14

x = ResNetBlock(512)(x) #Block_15
x = ResNetBlock(512)(x) #Block_16

x = layers.GlobalAveragePooling2D()(x)


outputs = Dense(7, activation = "softmax")(x)

model_resnet = keras.Model(inputs=inputs, outputs=outputs)
model_resnet.summary()

2024-07-14 14:36:54.217828: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 14:36:54.433874: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 14:36:54.433953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 14:36:54.438256: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 14:36:54.438333: I external/local_xla/xla/stream_executor

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block (ResNetBlock)     │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_1 (ResNetBlock)   │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_2 (ResNetBlock)   │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_3 (ResNetBlock)   │ (None, 28, 28, 128)    │       231,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_4 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_5 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_6 (ResNetBlock)   │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_7 (ResNetBlock)   │ (None, 14, 14, 256)    │       921,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_8 (ResNetBlock)   │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_9 (ResNetBlock)   │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_10 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_11 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_12 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_13 (ResNetBlock)  │ (None, 7, 7, 512)      │     3,677,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_14 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_15 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,313,799 (81.31 MB)

 Trainable params: 21,296,775 (81.24 MB)

 Non-trainable params: 17,024 (66.50 KB)

In [4]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [5]:
img_height = 224
img_width = 224
batch_size = 32

In [6]:
def load_images(path, limit=100):
    x = []
    y = []
    labels = os.listdir(path)
    for index, folder in enumerate(os.listdir(path)):
        lim = 0
        for image in os.listdir(path+"/"+folder):
            image_path = path+"/"+folder+"/"+image
            img = cv2.imread(image_path)
            img = np.array(img)
            x.append(img)
            y.append(index)
            lim+=1
            if lim > limit:
                break
    x = np.array(x)
    y = np.array(y)
    return x,y, labels

##### Creating training, validation dataset

In [7]:
x_train, y_train, labels_train = load_images("Data_Cancer/")
x_train = x_train/255.0

In [10]:
x_test, y_test, labels_test = load_images("Data_Cancer_Test/")
x_test = x_test/255.0

In [8]:
epochs = 50
model_resnet.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
history = model_resnet.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720960662.191334    1502 service.cc:145] XLA service 0x7fb768174910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720960662.191377    1502 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-07-14 14:37:42.624691: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-14 14:37:44.254683: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720960665.709122    1544 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8802', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1720960676.566760    1502 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.2512 - loss: 3.2777

I0000 00:00:1720960689.953899    1501 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_39', 160 bytes spill stores, 160 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_146', 48 bytes spill stores, 48 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 40s 606ms/step - accuracy: 0.2522 - loss: 3.2131
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.2492 - loss: 1.9356
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.2983 - loss: 1.8298
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - accuracy: 0.3654 - loss: 1.6402
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.3551 - loss: 1.6624
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.3677 - loss: 1.6641
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.3234 - loss: 1.6653
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.3688 - loss: 1.6302
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.3897 - loss: 1.6719
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.4052 - loss: 1.5307
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.4158 - loss: 1.5501
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accurac

In [13]:
model_vgg = Sequential([
    keras.layers.Input((224,224,3)),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(4096),
    Dense(4096),
    Dense(1000),
    
    Dense(7, activation = "softmax")
    
])
model_vgg.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_52 (Conv2D)              │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_64 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1000)           │     4,097,00

 Total params: 143,674,247 (548.07 MB)

 Trainable params: 143,674,247 (548.07 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
epochs = 50
model_vgg.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

history_vgg = model_vgg.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720961079.566151    2557 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 196 bytes spill stores, 196 bytes spill loads

I0000 00:00:1720961080.003762    2558 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 132 bytes spill stores, 132 bytes spill loads

I0000 00:00:1720961080.611785    2562 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 1068 bytes spill stores, 920 bytes spill loads

I0000 00:00:1720961080.630318    2564 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 504 bytes spill stores, 460 bytes spill loads

I0000 00:00:1720961080.903380    2557 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2681', 40 bytes spill stores, 40 bytes spill loads

I0000 00:00:172096108

22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.1415 - loss: 9.2543

I0000 00:00:1720961131.721657    2686 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 36 bytes spill stores, 36 bytes spill loads

I0000 00:00:1720961132.915063    2689 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 628 bytes spill stores, 580 bytes spill loads

I0000 00:00:1720961133.411659    2691 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1499', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 65s 755ms/step - accuracy: 0.1413 - loss: 8.8656
Epoch 2/50


I0000 00:00:1720961140.513958    1502 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_13', 164 bytes spill stores, 164 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_12', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 316ms/step - accuracy: 0.1410 - loss: 2.0294
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 311ms/step - accuracy: 0.1532 - loss: 1.9899
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 298ms/step - accuracy: 0.1535 - loss: 1.9663
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 299ms/step - accuracy: 0.1384 - loss: 1.9847
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 298ms/step - accuracy: 0.1321 - loss: 1.9605
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 298ms/step - accuracy: 0.1443 - loss: 1.9558
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 299ms/step - accuracy: 0.0995 - loss: 1.9883
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 308ms/step - accuracy: 0.1279 - loss: 1.9612
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 304ms/step - accuracy: 0.1491 - loss: 1.9834
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 306ms/step - accuracy: 0.1883 - loss: 1.9599
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 309ms/step - accuracy: 0.1157 - loss: 1.9627
Epoch 13/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 311ms/step - accurac

## Plain 34 Layer Model

In [23]:
model_34 = Sequential([
    Input((img_height, img_width, 3)),
    Conv2D(64, (7,7), strides = 2, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(128, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(256, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(512, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    GlobalAveragePooling2D(),
    Dense(1000),
    Dense(7, activation = "softmax")
    
    
])

model_34.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_134 (Conv2D)             │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_135 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_136 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_137 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_138 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_139 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_140 (Conv2D)             │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_141 (Conv2D)             │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_142 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_143 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_144 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_145 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_146 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_147 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_148 (Conv2D)             │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_149 (Conv2D)             │ (None, 14, 14, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_150 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_151 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_152 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_153 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_154 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_155 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_156 (Conv2D)             │ (None, 14, 14, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_157 (Conv2D)             │ (None, 14, 14, 256)    │       590,08

 Total params: 21,623,239 (82.49 MB)

 Trainable params: 21,623,239 (82.49 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
epochs = 50
model_34.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
history_34 = model_34.fit(x_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/50


I0000 00:00:1720961667.132254    3729 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 80 bytes spill stores, 80 bytes spill loads

I0000 00:00:1720961667.434467    3726 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 336 bytes spill stores, 336 bytes spill loads



22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.1310 - loss: 1.9515

I0000 00:00:1720961675.032960    3811 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2908', 100 bytes spill stores, 100 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 296ms/step - accuracy: 0.1294 - loss: 1.9516
Epoch 2/50
 1/23 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.1875 - loss: 1.9466

I0000 00:00:1720961678.131198    1504 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_32', 36 bytes spill stores, 36 bytes spill loads



23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.1458 - loss: 1.9489
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.1278 - loss: 1.9475
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.1443 - loss: 1.9468 
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.1673 - loss: 1.9468
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1428 - loss: 1.9468
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.1409 - loss: 1.9466 
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.1389 - loss: 1.9472
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.1478 - loss: 1.9461
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.1279 - loss: 1.9475
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.1210 - loss: 1.9470
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.1608 - loss: 1.9454 
Epoch 13/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.141

In [15]:
predictions = model_vgg.predict(x_test)
predictions = np.argmax(predictions, axis = 1)

tf.math.confusion_matrix(predictions, y_test)

16/17 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step

I0000 00:00:1720961511.758834    3634 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_210', 52 bytes spill stores, 52 bytes spill loads

I0000 00:00:1720961512.521323    3639 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_206', 52 bytes spill stores, 52 bytes spill loads

I0000 00:00:1720961512.665758    3641 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_214', 632 bytes spill stores, 584 bytes spill loads



17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 387ms/step


<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[  0,   0,   0,   0,   0,   0,   0],
       [ 43,  93, 101,  44, 101, 101,  35],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0]], dtype=int32)>

In [25]:
predictions = model_34.predict(x_test)
predictions = np.argmax(predictions, axis = 1)

tf.math.confusion_matrix(predictions, y_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step


<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[  0,   0,   0,   0,   0,   0,   0],
       [ 43,  93, 101,  44, 101, 101,  35],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0]], dtype=int32)>

In [11]:
predictions = model_resnet.predict(x_test)
predictions = np.argmax(predictions, axis = 1)

tf.math.confusion_matrix(predictions, y_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step


<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[11, 10, 11,  3,  2, 12,  2],
       [15, 39, 23, 21,  9, 10,  4],
       [ 0,  0,  3,  0,  0,  0,  0],
       [ 6, 11, 13,  9,  3,  7,  3],
       [ 8,  7, 21,  3, 50,  8,  4],
       [ 2,  6, 25,  6, 32, 60,  0],
       [ 1, 20,  5,  2,  5,  4, 22]], dtype=int32)>

In [ ]:
model_vgg.save("Models/vgg_19_model.keras")
model_34.save("Models/vgg_19_model.keras")
model_resnet.save("Models/resnet34_model.keras")